In [1]:
# import dependencies
from IPython.display import display, Javascript, Image, HTML
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import pandas as pd

In [2]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [3]:
# JavaScript y HTML para crear nuestro "front"
def video_stream():
  # 1. --- Definir el HTML y CSS de la interfaz ---
  ui_html = HTML('''
    <div style="font-family: sans-serif; padding: 10px; border: 1px solid #ccc; border-radius: 8px; width: 680px; margin: auto;">
        <h2 style="margin-top: 0;"> Detector de Movimiento</h2>
        <p>Haz clic en el video para detener la captura y guardar el CSV.</p>

        <div id="video_container" style="position: relative; width: 640px; height: 480px; margin: auto; border: 2px solid black;">
            <video id="video_element" style="display: block; width: 100%; height: 100%;" autoplay playsinline></video>
            <img id="overlay_element" style="position: absolute; top: 0; left: 0; width: 100%; height: 100%; z-index: 1;">
        </div>

        <div style="margin-top: 10px; font-size: 1.1em;">
            <strong>Estado:</strong> <span id="status_label" style="font-weight: bold; color: #555;">Iniciando...</span>
        </div>

        <div id="results_area" style="margin-top: 20px; text-align: center;">
            <p>Esperando la finalización del video para generar el análisis...</p>
        </div>
    </div>
  ''')
  display(ui_html)

  # 2. --- Definir el JavaScript (el "puente") ---
  # El código JS es el mismo que en la respuesta anterior (sin cambios)
  js_code = Javascript('''
    var video;
    var overlay;
    var captureCanvas;
    var labelElement;
    var stream;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       if (stream) {
           stream.getVideoTracks()[0].stop();
       }
       video = null;
       overlay = null;
       captureCanvas = null;
       labelElement = null;
       stream = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8);
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (video) {
        return stream;
      }

      // Encontrar los elementos que definimos en el HTML
      video = document.getElementById('video_element');
      overlay = document.getElementById('overlay_element');
      labelElement = document.getElementById('status_label');

      // Configurar el canvas oculto para la captura
      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640;
      captureCanvas.height = 480;

      // Iniciar la cámara
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment" }});
      video.srcObject = stream;
      await video.play();

      // Configurar el clic para detener
      video.onclick = () => { shutdown = true; };
      overlay.onclick = () => { shutdown = true; };

      window.requestAnimationFrame(onAnimationFrame);
      return stream;
    }

    // Esta es la función que Python llama (el "puente")
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        overlay.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')
  display(js_code)

# Esta es la función de Python que llama al JS "stream_frame"
def video_frame(label, bbox):
  data = eval_js('stream_frame(\"{}\", \"{}\")'.format(label, bbox))
  return data

In [4]:
# --- IMPORTACIONES ADICIONALES NECESARIAS PARA LA VISUALIZACIÓN ---
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from IPython.display import HTML
import io
import base64
import os
from flask import Flask, render_template_string
from threading import Thread
import time as time_module

# --- FUNCIÓN DE AYUDA PARA EXPORTAR GRÁFICAS A BASE64 ---
def plot_to_base64(fig, title=""):
    """Guarda una figura de Matplotlib en un string PNG base64 y la cierra."""
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight', dpi=150)
    plt.close(fig)
    data = base64.b64encode(buf.getbuffer()).decode("ascii")
    html_img = f'<div class="graph-container"><h4 class="graph-title">{title}</h4><img src="data:image/png;base64,{data}" class="graph-image expandable-image"></div>'
    return html_img

# --- INICIALIZACIÓN DEL BUCLE DE VIDEO (COMO ANTES) ---
video_stream()
label_html = 'Capturando...'
bbox = ''
count = 0
fgbg = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50, detectShadows=False)
centroid_data = []
start_time = time.time()
processed_frames = []

print("Iniciando stream... Haz clic en el video para detener y generar el análisis.")

# ----------------- BUCLE DE CAPTURA -----------------
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # Validar que js_reply tenga el formato correcto
    try:
        if "img" not in js_reply:
            print("Error: 'img' no encontrado en js_reply")
            break

        img = js_to_image(js_reply["img"])
        if img is None or img.size == 0:
            print("Error: Imagen vacía o inválida")
            continue

    except Exception as e:
        print(f"Error al procesar imagen: {e}")
        continue

    current_time = time.time() - start_time
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)
    display_img = img.copy()

    fgmask = fgbg.apply(img)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, kernel)
    cnts, _ = cv2.findContours(fgmask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    cx, cy = None, None
    if cnts:
        largest_contour = max(cnts, key=cv2.contourArea)
        if cv2.contourArea(largest_contour) > 100:
            M = cv2.moments(largest_contour)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                cv2.circle(bbox_array, (cx, cy), 10, (255, 0, 0, 255), -1)
                cv2.drawContours(bbox_array, [largest_contour], -1, (0, 255, 0, 255), 2)
                cv2.circle(display_img, (cx, cy), 10, (0, 0, 255), -1)
                cv2.drawContours(display_img, [largest_contour], -1, (0, 255, 0), 2)

    centroid_data.append([count, current_time, cx, cy])
    processed_frames.append(display_img.copy())

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    bbox_bytes = bbox_to_bytes(bbox_array)
    bbox = bbox_bytes

    label_html = f"Procesando frame {count}..."
    count += 1

print(f"\nStream detenido. Se procesaron {count} fotogramas.")

# --- 1. GUARDAR CSV Y PREPARAR ANÁLISIS ---
csv_filename = 'centroid_data.csv'
df = pd.DataFrame(centroid_data, columns=['frame_index', 'time_seconds', 'centroid_x', 'centroid_y'])
df.to_csv(csv_filename, index=False)
print(f"Datos guardados exitosamente en '{csv_filename}'")

right_panel_content = ''
video_and_coeff_content = ''
coeff_section_html = ''

df_filtered = df.dropna(subset=['centroid_x', 'centroid_y']).reset_index(drop=True)

# ----------------- 2. GENERACIÓN DE GRÁFICAS Y TEXTO -----------------
if not df_filtered.empty:
    x_coords = df_filtered['centroid_x']
    y_coords = df_filtered['centroid_y']
    time_coords = df_filtered['time_seconds']

    # A. Gráfico 1: Trayectoria X vs Y
    fig1, ax1 = plt.subplots(figsize=(10, 7))
    ax1.plot(x_coords, y_coords, marker='o', markersize=5, linestyle='-', linewidth=2, color='#1a73e8')
    ax1.set_title("Trayectoria del centroide (X vs Y)", fontsize=14, fontweight='500', pad=15)
    ax1.set_xlabel("Coordenada X (píxeles)", fontsize=12)
    ax1.set_ylabel("Coordenada Y (píxeles)", fontsize=12)
    ax1.grid(True, alpha=0.3, linestyle='--', linewidth=0.8)
    ax1.invert_yaxis()
    ax1.tick_params(labelsize=10)
    plt.tight_layout()
    right_panel_content += plot_to_base64(fig1, "1. Trayectoria del Centroide")

    # B. Gráfico 2: Y vs Tiempo y Picos de Rebote
    fig2, ax2 = plt.subplots(figsize=(10, 7))
    ax2.plot(time_coords, y_coords, label="Trayectoria Y vs Tiempo", color="#1a73e8", linewidth=2)

    inverted_disp_array = y_coords.values * -1
    inverted_peaks, _ = find_peaks(inverted_disp_array, distance=10, prominence=5)

    if len(inverted_peaks) > 0:
        ax2.scatter(time_coords.values[inverted_peaks], y_coords.values[inverted_peaks],
                   color="red", s=80, zorder=3, label="Picos detectados (Rebotes)", marker='^')

    ax2.set_title("2. Coordenada Y en Función del Tiempo (con Rebotes)", fontsize=14, fontweight='500', pad=15)
    ax2.set_xlabel("Tiempo (s)", fontsize=12)
    ax2.set_ylabel("Coordenada Y (píxeles)", fontsize=12)
    ax2.legend(fontsize=11, loc='best', framealpha=0.9)
    ax2.grid(True, alpha=0.3, linestyle='--', linewidth=0.8)
    ax2.invert_yaxis()
    ax2.tick_params(labelsize=10)
    plt.tight_layout()
    right_panel_content += plot_to_base64(fig2, "2. Posición Y vs Tiempo")

    # C. Cálculo de Coeficiente de Restitución (e)
    if len(inverted_peaks) > 1:
        peaks_y = np.insert(y_coords.values[inverted_peaks], 0, y_coords.iloc[0])
        y_floor = np.max(y_coords)
        heights = np.array([y_floor - peak for peak in peaks_y])
        h_drops = heights[:-1]
        h_rebounds = heights[1:]
        h_drops[h_drops <= 0] = 1e-6

        restitution_coeffs = np.sqrt(h_rebounds / h_drops)
        avg_coeff = restitution_coeffs.mean()

        elasticity_classification = ""
        if avg_coeff >= 0.95:
            elasticity_classification = "Elástico"
        elif avg_coeff <= 0.05:
            elasticity_classification = "Inelástico"
        else:
            elasticity_classification = "Parcialmente Elástico"

        coeff_list_formatted = ', '.join([f'{c:.4f}' for c in restitution_coeffs])

        coeff_section_html = f'''
        <div class="coefficient-section">
            <h4 class="coeff-title">3. Coeficiente de Restitución (e)</h4>
            <div class="coeff-item">
                <span class="coeff-label">Rebotes detectados:</span>
                <span class="coeff-value">{len(restitution_coeffs)}</span>
            </div>
            <div class="coeff-item">
                <span class="coeff-label">Coeficientes calculados (e):</span>
                <span class="coeff-value-list">{coeff_list_formatted}</span>
            </div>
            <div class="coeff-item-highlight">
                <span class="coeff-label">Coeficiente de restitución promedio:</span>
                <span class="coeff-value-highlight">{avg_coeff:.4f}</span>
            </div>
            <div class="coeff-item">
                <span class="coeff-label">Clasificación del objeto:</span>
                <span class="coeff-classification">{elasticity_classification}</span>
            </div>
        </div>

        <div class="info-section">
            <h4 class="info-title"> ¿Qué es el Coeficiente de Restitución?</h4>
            <p class="info-text">
                El coeficiente de restitución (e) mide la <strong>elasticidad de un objeto</strong> al rebotar.
                Compara la altura alcanzada después del rebote con la altura de caída inicial.
            </p>
            <ul class="info-list">
                <li><strong>e ≈ 1.0 (Elástico):</strong> El objeto recupera casi toda su energía. Ejemplo: pelota de golf, superball.</li>
                <li><strong>e ≈ 0.5 (Parcialmente Elástico):</strong> El objeto pierde energía moderada. Ejemplo: pelota de tenis, baloncesto.</li>
                <li><strong>e ≈ 0.0 (Inelástico):</strong> El objeto no rebota, absorbiendo toda la energía. Ejemplo: masa de plastilina.</li>
            </ul>
            <p class="info-text">
                <strong>Aplicaciones:</strong> Diseño de equipos deportivos, análisis de colisiones vehiculares,
                ingeniería de materiales y control de calidad en manufactura.
            </p>
        </div>
        '''
    else:
        coeff_section_html = '<div class="error-message">No se detectaron suficientes rebotes (mínimo 2) para calcular el coeficiente de restitución (e) y clasificar el objeto.</div>'
else:
    right_panel_content += '<div class="error-message">No se detectó ningún movimiento válido para generar las gráficas.</div>'
    coeff_section_html = '<div class="error-message">No se detectó ningún movimiento válido para calcular el coeficiente.</div>'

# --- GENERAR EL VIDEO A PARTIR DE LOS FOTOGRAMAS PROCESADOS ---
video_filename = 'motion_detection_video.mp4'

if processed_frames:
    height, width, _ = processed_frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = 10

    if os.path.exists(video_filename):
        os.remove(video_filename)
        print(f"Archivo '{video_filename}' existente eliminado.")

    out = cv2.VideoWriter(video_filename, fourcc, fps, (width, height))

    for frame in processed_frames:
        out.write(frame)

    out.release()
    print(f"Video '{video_filename}' generado exitosamente con {len(processed_frames)} fotogramas.")

    with open(video_filename, 'rb') as f:
        video_bytes = f.read()
    video_base64 = base64.b64encode(video_bytes).decode('ascii')

    try:
        with open('logo.png', 'rb') as f:
            logo_bytes = f.read()
        logo_base64 = base64.b64encode(logo_bytes).decode('ascii')
        logo_html = f'<img src="data:image/png;base64,{logo_base64}" alt="TrajecPro Logo" class="logo">'
    except FileNotFoundError:
        logo_html = ''
        print("Advertencia: 'logo.png' no encontrado. El logo no se mostrará.")

    video_and_coeff_content = f'''
    <div class="left-header">
        <h2 class="main-title">{logo_html}<span class="title-text">TrajecPro</span></h2>
    </div>
    <div class="video-container">
        <video controls autoplay muted loop class="video-player">
            <source src="data:video/mp4;base64,{video_base64}" type="video/mp4">
            Tu navegador no soporta el tag de video.
        </video>
        <p class="video-caption">Este video muestra el objeto detectado y su centroide en cada fotograma.</p>
    </div>
    ''' + coeff_section_html
else:
    print("No se capturaron fotogramas para generar el video.")
    video_and_coeff_content = '<div class="error-message">No se pudo generar el video de detección de movimiento.</div>' + coeff_section_html

# ----------------- 3. INYECTAR RESULTADOS COMBINADOS AL FRONTEND CON NUEVA VISUALIZACIÓN -----------------

website_style = '''
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, 'Helvetica Neue', Arial, sans-serif;
            background-color: #ffffff;
            color: #202124;
            line-height: 1.5;
        }

        .main-container {
            display: grid;
            grid-template-columns: 380px 1fr;
            max-width: 1400px;
            margin: 0 auto;
            background-color: #ffffff;
            min-height: 100vh;
        }

        .left-panel {
            background-color: #ffffff;
            padding: 24px;
            border-right: 1px solid #e8eaed;
        }

        .right-panel {
            background-color: #fafafa;
            padding: 32px 40px;
        }

        /* Left Panel Styles */
        .left-header {
            margin-bottom: 24px;
        }

        .main-title {
            font-size: 24px;
            font-weight: 400;
            color: #202124;
            display: flex;
            align-items: center;
            margin-bottom: 0;
            border-bottom: none;
            padding-bottom: 0;
        }

        .logo {
            height: 28px;
            margin-right: 12px;
            vertical-align: middle;
        }

        .title-text {
            font-weight: 400;
        }

        .video-container {
            margin-bottom: 24px;
        }

        .video-player {
            width: 100%;
            height: auto;
            border-radius: 8px;
            background-color: #000;
            display: block;
            box-shadow: 0 1px 3px rgba(0,0,0,0.12);
        }

        .video-caption {
            font-size: 12px;
            color: #5f6368;
            text-align: center;
            margin-top: 12px;
            line-height: 1.4;
        }

        /* Coefficient Section */
        .coefficient-section {
            background-color: #e8f7fc;
            border-left: 4px solid #91f1ff;
            padding: 16px 20px;
            border-radius: 4px;
            margin-top: 16px;
        }

        .coeff-title {
            font-size: 16px;
            font-weight: 500;
            color: #0288d1;
            margin-bottom: 16px;
            border-bottom: none;
            padding-bottom: 0;
        }

        .coeff-item {
            margin-bottom: 12px;
            font-size: 13px;
            line-height: 1.6;
        }

        .coeff-item:last-child {
            margin-bottom: 0;
        }

        .coeff-item-highlight {
            background-color: #f3e5f5;
            padding: 12px;
            border-radius: 4px;
            margin-bottom: 12px;
            border-left: 3px solid #9956ca;
        }

        .coeff-label {
            color: #01579b;
            font-weight: 500;
            display: block;
            margin-bottom: 4px;
        }

        .coeff-value {
            color: #0277bd;
            font-weight: 400;
            display: block;
        }

        .coeff-value-list {
            color: #0277bd;
            font-weight: 400;
            display: block;
            word-break: break-word;
        }

        .coeff-value-highlight {
            color: #9956ca;
            font-weight: 700;
            font-size: 16px;
            display: block;
        }

        .coeff-classification {
            color: #1967d2;
            font-weight: 600;
            font-size: 15px;
            display: block;
        }

        /* Right Panel Styles */
        .right-panel h3 {
            font-size: 20px;
            font-weight: 500;
            color: #202124;
            margin-bottom: 24px;
            border-bottom: none;
            padding-bottom: 0;
        }

        .graph-container {
            background-color: #ffffff;
            border-radius: 8px;
            padding: 20px;
            margin-bottom: 24px;
            box-shadow: 0 1px 3px rgba(0,0,0,0.08);
        }

        .graph-title {
            font-size: 15px;
            font-weight: 500;
            color: #202124;
            margin-bottom: 16px;
            border-bottom: none;
            padding-bottom: 0;
        }

        .graph-image {
            width: 100%;
            height: auto;
            display: block;
            border-radius: 4px;
            cursor: pointer;
            transition: transform 0.2s ease;
        }

        .graph-image:hover {
            transform: scale(1.02);
            box-shadow: 0 4px 12px rgba(0,0,0,0.15);
        }

        /* Modal para expandir imágenes */
        .image-modal {
            display: none;
            position: fixed;
            z-index: 9999;
            left: 0;
            top: 0;
            width: 100%;
            height: 100%;
            background-color: rgba(0, 0, 0, 0.95);
            overflow: auto;
            animation: fadeIn 0.3s;
        }

        @keyframes fadeIn {
            from { opacity: 0; }
            to { opacity: 1; }
        }

        .image-modal-content {
            display: block;
            margin: 2% auto;
            max-width: 95%;
            max-height: 95%;
            object-fit: contain;
            animation: zoomIn 0.3s;
        }

        @keyframes zoomIn {
            from { transform: scale(0.8); }
            to { transform: scale(1); }
        }

        .image-modal-close {
            position: absolute;
            top: 20px;
            right: 40px;
            color: #fff;
            font-size: 40px;
            font-weight: bold;
            cursor: pointer;
            transition: 0.3s;
            z-index: 10000;
        }

        .image-modal-close:hover,
        .image-modal-close:focus {
            color: #bbb;
        }

        .image-modal-caption {
            text-align: center;
            color: #ccc;
            padding: 20px;
            font-size: 18px;
        }

        .error-message {
            background-color: #fce8e6;
            border-left: 4px solid #d93025;
            padding: 16px;
            color: #c5221f;
            font-size: 13px;
            border-radius: 4px;
            margin-top: 16px;
        }

        /* Responsive Design */
        @media (max-width: 1024px) {
            .main-container {
                grid-template-columns: 1fr;
            }

            .left-panel {
                border-right: none;
                border-bottom: 1px solid #e8eaed;
                max-width: 100%;
            }

            .right-panel {
                padding: 24px;
            }
        }

        @media (max-width: 768px) {
            .left-panel, .right-panel {
                padding: 16px;
            }

            .main-title {
                font-size: 20px;
            }

            .right-panel h3 {
                font-size: 18px;
            }
        }
    </style>
'''

final_combined_html = f'''
    {website_style}
    <!-- Modal para expandir imágenes -->
    <div id="imageModal" class="image-modal">
        <span class="image-modal-close">&times;</span>
        <img class="image-modal-content" id="modalImage">
        <div class="image-modal-caption" id="modalCaption"></div>
    </div>

    <div class="main-container">
        <div class="left-panel">
            {video_and_coeff_content}
        </div>
        <div class="right-panel">
            <h3>Análisis de Resultados de Trayectoria y Posición</h3>
            {right_panel_content}
        </div>
    </div>

    <script>
        // Funcionalidad del modal para expandir imágenes
        document.addEventListener('DOMContentLoaded', function() {{
            const modal = document.getElementById('imageModal');
            const modalImg = document.getElementById('modalImage');
            const modalCaption = document.getElementById('modalCaption');
            const closeBtn = document.querySelector('.image-modal-close');

            // Agregar evento click a todas las imágenes expandibles
            const expandableImages = document.querySelectorAll('.expandable-image');
            expandableImages.forEach(function(img) {{
                img.addEventListener('click', function() {{
                    modal.style.display = 'block';
                    modalImg.src = this.src;
                    const title = this.closest('.graph-container').querySelector('.graph-title');
                    modalCaption.textContent = title ? title.textContent : '';
                }});
            }});

            // Cerrar modal al hacer clic en X
            closeBtn.addEventListener('click', function() {{
                modal.style.display = 'none';
            }});

            // Cerrar modal al hacer clic fuera de la imagen
            modal.addEventListener('click', function(e) {{
                if (e.target === modal) {{
                    modal.style.display = 'none';
                }}
            }});

            // Cerrar modal con tecla ESC
            document.addEventListener('keydown', function(e) {{
                if (e.key === 'Escape' && modal.style.display === 'block') {{
                    modal.style.display = 'none';
                }}
            }});
        }});
    </script>
'''

final_display_js_combined = f'''
    var results_area = document.getElementById('results_area');
    if (results_area) {{
        results_area.innerHTML = `{final_combined_html.replace("`", "\\`")}`;
        var parent_div = results_area.closest('div[style]');
        if (parent_div) {{
            parent_div.removeAttribute('style');
        }}
    }}
'''

eval_js(final_display_js_combined)

<IPython.core.display.Javascript object>

Iniciando stream... Haz clic en el video para detener y generar el análisis.


/tmp/ipython-input-2186210051.py:27: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')



Stream detenido. Se procesaron 82 fotogramas.
Datos guardados exitosamente en 'centroid_data.csv'
Video 'motion_detection_video.mp4' generado exitosamente con 82 fotogramas.
Advertencia: 'logo.png' no encontrado. El logo no se mostrará.


In [5]:
# ----------------- 4. CREAR SERVIDOR WEB Y LINK PÚBLICO -----------------

# Guardar el HTML completo en un archivo
html_output_file = 'trajecpro_results.html'
with open(html_output_file, 'w', encoding='utf-8') as f:
    f.write(f'''
    <!DOCTYPE html>
    <html lang="es">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>TrajecPro - Análisis de Trayectoria</title>
    </head>
    <body>
        {final_combined_html}
    </body>
    </html>
    ''')

print(f"\n Archivo HTML guardado: {html_output_file}")

# Importar Flask (corrección del error)
from flask import Flask, render_template_string
from threading import Thread
import time as time_module

# Crear aplicación Flask
app = Flask(__name__)

@app.route('/')
def home():
    with open(html_output_file, 'r', encoding='utf-8') as f:
        html_content = f.read()
    return html_content

def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# Iniciar servidor Flask en un hilo separado
flask_thread = Thread(target=run_flask, daemon=True)
flask_thread.start()

print("\n Iniciando servidor Flask...")
time_module.sleep(3)  # Dar tiempo para que Flask inicie

# Instalar e iniciar ngrok
print("\n Instalando ngrok...")
get_ipython().system('pip install -q pyngrok')

from pyngrok import ngrok

# --- IMPORTANTE: Configura tu token de autenticación de ngrok aquí ---
# Si ya tienes una cuenta de ngrok, obtén tu token desde https://dashboard.ngrok.com/get-started/your-authtoken
# y reemplaza "TU_AUTHTOKEN_AQUI" con tu token real.

# ngrok.set_auth_token("TU_AUTHTOKEN_AQUI")
# Descomenta la línea de arriba y pega tu token si tienes uno.

# Crear túnel público
print("\n Creando túnel público con ngrok...")
public_url = ngrok.connect(5000, bind_tls=True)
ngrok_url = public_url.public_url

print("\n" + "="*60)
print(" ¡LISTO! Tu análisis está disponible públicamente:")
print("="*60)
print(f"\n Link público: {ngrok_url}")
print(f"\n Comparte este link para ver los resultados desde cualquier dispositivo")
print("\n  Nota: El link estará activo mientras este notebook esté ejecutándose")
print("="*60)

# Mantener el servidor activo
print("\n Servidor activo. Presiona el botón 'Stop' para detener.")
print(" El link seguirá funcionando hasta que detengas esta celda.\n")


 Archivo HTML guardado: trajecpro_results.html

 Iniciando servidor Flask...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit



 Instalando ngrok...

 Creando túnel público con ngrok...

 ¡LISTO! Tu análisis está disponible públicamente:

 Link público: https://kai-unwhimpering-unseeingly.ngrok-free.dev

 Comparte este link para ver los resultados desde cualquier dispositivo

  Nota: El link estará activo mientras este notebook esté ejecutándose

 Servidor activo. Presiona el botón 'Stop' para detener.
 El link seguirá funcionando hasta que detengas esta celda.



In [ ]:
"""
import os
from pyngrok import ngrok

# 1. Kill all ngrok tunnels first
try:
    ngrok.kill()
    print(" Ngrok processes killed.")
except:
    pass

# 2. Function to find and kill processes on specific ports
def kill_port(port):
    try:
        # 'fuser -k' finds the process using the port and kills it
        result = os.system(f"fuser -k {port}/tcp")
        if result == 0:
            print(f" Process on port {port} killed successfully.")
        else:
            print(f"ℹ No process found on port {port} (or failed to kill).")

        # Fail-safe: Try using lsof just in case fuser didn't catch it
        os.system(f"lsof -t -i:{port} | xargs kill -9 2>/dev/null")
    except Exception as e:
        print(f"Error cleaning port {port}: {e}")

# 3. Kill the common ports we've been using
kill_port(5000)  # Default Flask port
kill_port(5050)  # The port used in the V5 code
"""

'\nimport os\nfrom pyngrok import ngrok\n\n# 1. Kill all ngrok tunnels first\ntry:\n    ngrok.kill()\n    print("✅ Ngrok processes killed.")\nexcept:\n    pass\n\n# 2. Function to find and kill processes on specific ports\ndef kill_port(port):\n    try:\n        # \'fuser -k\' finds the process using the port and kills it\n        result = os.system(f"fuser -k {port}/tcp")\n        if result == 0:\n            print(f"✅ Process on port {port} killed successfully.")\n        else:\n            print(f"ℹ️ No process found on port {port} (or failed to kill).")\n            \n        # Fail-safe: Try using lsof just in case fuser didn\'t catch it\n        os.system(f"lsof -t -i:{port} | xargs kill -9 2>/dev/null")\n    except Exception as e:\n        print(f"Error cleaning port {port}: {e}")\n\n# 3. Kill the common ports we\'ve been using\nkill_port(5000)  # Default Flask port\nkill_port(5050)  # The port used in the V5 code\n'